In [ ]:
#default_exp logger

# Logger

> The logger is the widget that allows us to log errors in a clean way.

It can be used anywhere in the notebook development environment or the application. Logs can be visualized by displaying the WidgetHandler. This widget is tucked away in the "Settings" tab of the application, but can be seen anywhere by displaying the WIDGET_HANDLER object. All copies of the widget throughout these notebooks will display the same logs, even if they were handled before the widget was displayed.

## Configuration Model

In [ ]:
from pathlib import Path
import os, pathlib, urllib, configparser
import logging

In [ ]:
#export
class ConfigModel(configparser.ConfigParser):
    
    def __init__(self):
        
        ''' a custom converter we use to get a log level value from it's string representation '''
        converters={'loglevel': lambda string: getattr(logging, string)}
        super().__init__(converters=converters)
        
        ''' root directory of repository '''
        bin = pathlib.Path().absolute().parent
        
        ''' user paths '''
        self.user_dir = Path('~/.nbdev_app_template').expanduser()
        self.user_dir.mkdir(exist_ok=True)
        self.user_config = self.user_dir / 'config.ini'
        
        ''' default config '''
        defaults = {'LOG': {'level': 'INFO',
                            'mode': 'w',
                            'captureWarnings': True,
                            'filename': self.user_dir / 'superpower.log'},
                    'ERRORS': {'catch_all': True}
                   }
        self.read_dict(defaults)
        
        ''' read and validate user config, which overrides default '''
        if self.user_config.is_file:
            try:
                self.read(self.user_config)
            except Exception as e:
                pass
        
        ''' remove old configs'''
        self.write_user_config()
        
    def write_user_config(self):
        with open(self.user_config, 'w') as file:
            self.write(file)
            
    def _repr_pretty_(self, p, cycle):
            for section in self.sections():
                for key, value in self[section].items():
                    p.text(key + ': ')
                    p.pretty(value)
                    p.breakable()

In [ ]:
#export
CONFIG = ConfigModel()

In [ ]:
from traitlets import HasTraits, Unicode, Bool, observe

In [ ]:
#exports
class SettingsModel(HasTraits):
    
    logLevel = Unicode()
    catchAll = Bool()

    def __init__(self, **kwargs):
        super().__init__(**kwargs)

        self.logLevel = CONFIG['LOG']['level']
        self.catchAll = CONFIG['ERRORS'].getboolean('catch_all')

    @observe('logLevel')
    def observeLogLevel(self, change):
        loggers = [logging.getLogger(name) for name in logging.root.manager.loggerDict]
        for logger in loggers:
            logger.setLevel(change['new'])
        CONFIG['LOG']['level'] = change['new']
        CONFIG.write_user_config()
       
    @observe('catchAll')
    def observeCatchAll(self, change):
        CONFIG['ERRORS']['catch_all'] = str(change['new'])
        CONFIG.write_user_config()

Settings are persistent across sessions and written to our config file when changed. For development, we probably want the following settings:

In [ ]:
model = SettingsModel()
model.logLevel = 'INFO'
model.catchAll = False

In [ ]:
#export
import ipywidgets as ipyw
import traitlets
import warnings
import os, pathlib, urllib
from IPython.display import HTML, Javascript, display
from pathlib import Path
import logging

The DispatchingFormatter allows us to define many log formats that are responsive to the name of the created logger.

In [ ]:
#exports
class DispatchingFormatter:
    """Dispatch formatter for logger and it's sub logger."""
    def __init__(self):
        self.formatters = {
            'root': logging.Formatter('%(message)s'),
            'r.console': logging.Formatter('[R.%(levelname)s] %(message)s'),
            'py.warnings': logging.Formatter('[PY.WARNINGS] %(message)s'),
            '__main__': logging.Formatter('[PY.%(levelname)s] %(message)s'),
            'default': logging.Formatter('[PY.%(levelname)s](%(name)s:%(lineno)d)  %(message)s')
        }

    def format(self, record):
        formatter = self.formatters.get(record.name, self.formatters['default'])
        return formatter.format(record)

In [ ]:
#export
FORMATTER = DispatchingFormatter()

Notice that the name of the root logger is an empty string. Settings applied to the root logger are applied to all loggers.

In [ ]:
#export
ROOT_LOGGER = logging.getLogger('')
ROOT_LOGGER.setLevel(CONFIG['LOG']['level'])

This will cause a log file to be created in the user's superpower_gui directory.

In [ ]:
#export 
FILE_HANDLER = logging.FileHandler(CONFIG['LOG']['filename'],
                               CONFIG['LOG']['mode'])
FILE_HANDLER.setFormatter(FORMATTER)
logging.getLogger('').addHandler(FILE_HANDLER)

The `WidgetHandler` is handles log messages by displaying them in a Textarea widget. This widget is tucked away in the "Settings" tab of the app.

In [ ]:
#exports
class WidgetHandler(ipyw.Textarea, logging.Handler):
    
    def __init__(self, config):
        super().__init__(layout={'width': '100%', 'height': '250px'}, disabled=True)
        self.config = config
        
    def emit(self, record):
        self.value += str(self.format(record)) + '\n'

In [ ]:
#export
WIDGET_HANDLER = WidgetHandler(CONFIG)
WIDGET_HANDLER.setFormatter(FORMATTER)
logging.getLogger('').addHandler(WIDGET_HANDLER)

In [ ]:
WIDGET_HANDLER

WidgetHandler(value='', disabled=True, layout=Layout(height='250px', width='100%'))

We can use this function in any notebook to log to the displayed copy of the WIDGET_HANDLER. We use this function because we don't want to import a fresh copy of `logging` a different module.

In [ ]:
#export
def getLogger(name=''):
    return logging.getLogger(name)

In [ ]:
#export
logger = getLogger()
logger.info('Welcome to Superpower!')

Many python packages used in this application throw warnings. We want to caputure those warnings in the log so they don't display unwanted messages to the screen.

In [ ]:
warnings.warn("Not caputured")

/var/folders/4m/nw6z_yqn4jx4gts24l23w25h0000gn/T/ipykernel_3926/2973509925.py:1: UserWarning: Not caputured
  warnings.warn("Not caputured")


In [ ]:
#export 
logging.captureWarnings(CONFIG['LOG']['captureWarnings'])

In [ ]:
warnings.warn("Once we create the handler, the warning will be captured")

In [ ]:
#hide
from nbdev.export import notebook2script; notebook2script()

MissingSectionHeaderError: File contains no section headers.
file: Path('/Users/nicolebrewer/Repos/nicole-brewer/nbdev_app_template/settings.ini'), line: 1
'DEFAULT]\n'